In [ ]:
!pip install rtree networkx sklearn osmium pysqlite3 shapely

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached Rtree-1.0.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.0 MB)
  Using cached sklearn-0.0.post4.tar.gz (3.6 kB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sklearn: filename=sklearn-0.0.post4-py3-none-any.whl size=2973 sha256=fd3cee1c5cec11ff553897b71a4df4400ad1a4d7941bcea525a00a04bebd726a
  Stored in directory: /root/.cache/pip/wheels/46/b8/69/4f7789b7d2fc0718a44cbc4ae5d40970a40c22c472ae50f0a1
  Created wheel for pysqlite3: filename=pysqlite3-0.5.0-cp310-cp310-linux_x86_64.whl size=162327 sha256=b3ea796a44760201107788cbb6749d472debc5d57f950ee8db9b3b40078f78c1
  Stored in directory: /root/.cache/pip/wheels/63/4e/34/5a90d9770b5fbf518f7a5da144c55

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
directory = '/content/drive/MyDrive/CS541Project/'
fileName = '/content/drive/MyDrive/CS541Project/data/West_Lafayette.osm.pbf'
#filename = '/content/drive/MyDrive/CS541Project/planet_-86.983,40.412_-86.863,40.477.osm'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import osmium
import math
from shapely.geometry import Point, LineString
from rtree import index


class POIHandler(osmium.SimpleHandler):
    def __init__(self):
        osmium.SimpleHandler.__init__(self)
        self.tags = ['name','amenity','shop']
        self.wayTags = ['highway']
        self.nodes = {}
        self.edges = []
        self.poi = set()
        self.index = 0
        self.rtree_edge = index.Index()
    def haversine_distance(self,lat1, lon1, lat2, lon2):
      R = 3958.7613  # Earth's radius in miles
      lat1_rad = math.radians(lat1)
      lon1_rad = math.radians(lon1)
      lat2_rad = math.radians(lat2)
      lon2_rad = math.radians(lon2)
      dlat = lat2_rad - lat1_rad
      dlon = lon2_rad - lon1_rad
      a = math.sin(dlat/2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon/2)**2
      c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
      distance = R * c  # Distance in miles
      return distance
    def node(self, n):
        self.nodes[n.id] = (n.location.lon, n.location.lat)
        node_tags = {tag.k: tag.v for tag in n.tags if tag.k in self.tags}
        # ID,highway,amenity,name,shop
        if len(node_tags) !=0:
          node_data = [n.id]
          for tag in self.tags:
            if tag in node_tags:
              node_data.append(node_tags[tag])
            else:
              node_data.append('')
            
          self.poi.add(tuple(node_data))

    def way(self, w):
        highway = ""
        line_list = []
        for tag in w.tags:
          if tag.k in self.wayTags:
            highway = tag.v
        for i in range(len(w.nodes) - 1):
            node1 = w.nodes[i].ref
            node2 = w.nodes[i + 1].ref

            if node1 in self.nodes and node2 in self.nodes:
                line_list.append(list(self.nodes[node1]))
                line_list.append(list(self.nodes[node2]))
                lon1, lat1 = self.nodes[node1]
                lon2, lat2 = self.nodes[node2]
                distance = self.haversine_distance(lat1, lon1, lat2, lon2)

                self.edges.append((self.index,w.id,node1, node2, distance,highway))
                self.index +=1
        if len(line_list) >0:
            self.rtree_edge.insert(w.id, LineString(line_list).bounds)

# x = POIHandler()
# x.apply_file(fileName)
# print(len(list(x.rtree_edge.intersection((-86.983,40.412,-86.863,40.477)))))

In [ ]:
import sqlite3

class OSMDatabase:
    def __init__(self, db_file, osm_file, handler):
        self.conn = sqlite3.connect(db_file)
        self.cursor = self.conn.cursor()
        self.handler = handler
        self.handler.apply_file(osm_file)
        self.rtree = self.handler.rtree_edge
        self.create_tables()
        self.fetch_data()

    def build_poi_query(self):
        return "INSERT INTO POI (ID, NAME, AMENITY, SHOP) VALUES (?, ?, ?, ?)"

    def build_node_query(self):
        return "INSERT INTO NODES (ID, LON, LAT) VALUES (?, ?, ?)"

    def build_edge_query(self):
        return "INSERT INTO EDGES (ID, WID,START, END, DISTANCE, HIGHWAY) VALUES (?, ?, ?, ?, ?,?)"

    def fetch_nodes(self):
        for key, value in self.handler.nodes.items():
            self.cursor.execute(self.build_node_query(), (key, value[0], value[1]))

    def fetch_poi(self):
        for i in self.handler.poi:
            self.cursor.execute(self.build_poi_query(), i)

    def fetch_edges(self):
        for edge in self.handler.edges:
            self.cursor.execute(self.build_edge_query(), edge)

    def create_poi_table(self):
        try:
            self.cursor.execute("CREATE TABLE POI  \
                                  (ID BIGINT PRIMARY KEY NOT NULL,\
                                   NAME TEXT,\
                                   AMENITY CHAR(50),\
                                   SHOP CHAR(50));")
        except Exception as e:
            print(str(e))

    def create_nodes_table(self):
        try:
            self.cursor.execute("CREATE TABLE NODES  \
                                  (ID BIGINT PRIMARY KEY NOT NULL,\
                                   LON REAL,\
                                   LAT REAL);")
        except Exception as e:
            print(str(e))

    def create_edges_table(self):
        try:
            self.cursor.execute("CREATE TABLE EDGES \
                                  (ID BIGINT PRIMARY KEY NOT NULL,\
                                   WID BIGINT,\
                                   START BIGINT,\
                                   END BIGINT,\
                                   DISTANCE REAL,\
                                   HIGHWAY CHAR(50));")
        except Exception as e:
            print(str(e))

    def create_tables(self):
        self.create_edges_table()
        self.create_nodes_table()
        self.create_poi_table()

    def fetch_data(self):
        self.fetch_nodes()
        self.fetch_poi()
        self.fetch_edges()

    def show_all_data(self, table_name):
        query = "SELECT * FROM {}".format(table_name)
        self.cursor.execute(query)
        rows = self.cursor.fetchall()
        return rows
    def get_by_id(self,table_name,id):
        query = "SELECT * FROM {} WHERE ID = {}".format(table_name,id)
        self.cursor.execute(query)
        rows = self.cursor.fetchall()
        return rows[0]

    def get_id_by_coord(self,table_name,lon,lat):
        query = "SELECT * FROM {} WHERE LON = {} and LAT = {}".format(table_name,lon,lat)
        self.cursor.execute(query)
        rows = self.cursor.fetchall()
        return rows[0]

    def get_nodes_from_edge(self,table_name,wayid):
        query = "SELECT * FROM {} WHERE WID = {}".format(table_name,wayid)
        self.cursor.execute(query)
        rows = self.cursor.fetchall()
        temp = set()
        for edge_id, wid, start, end, distance, highway in rows:
          temp.add(start)
          temp.add(end)
        return list(temp)
    def close_connection(self):
        self.conn.close()

# Example usage
all_info = POIHandler()
db = OSMDatabase("one.db", fileName, all_info)
nodes = db.show_all_data("NODES")
edges = db.show_all_data("EDGES")
poi = db.show_all_data("POI")
# db.close_connection()

In [ ]:


import networkx as nx

class Node:
    def __init__(self, node_id, lon, lat):
        self.id = node_id
        self.lat = lat
        self.lon = lon

    def __str__(self):
        return f"Node({self.id}, {self.lon}, {self.lat})"

def create_graph(nodes, edges):
    G = nx.Graph()
    
    for node_id, lon , lat in nodes:
        G.add_node(node_id, data=Node(node_id, lon, lat))
    print(len(nodes))
    for edge_id, wid, start, end, distance, highway in edges:
        G.add_edge(start, end, weight=distance)
    print(len(edges))
    return G

def shortest_path(G, start_id, end_id):
    try:
        path = nx.shortest_path(G, start_id, end_id, weight='weight')
        nodes = [G.nodes[node]['data'] for node in path]
        return path
    except nx.NetworkXNoPath:
        print("No path found between the given nodes.")
        return []

G = create_graph(nodes, edges)
start_node_id = 358665942
end_node_id = 9519034907  
start_node = db.get_by_id('NODES',start_node_id)
end_node = db.get_by_id('NODES',end_node_id)

start_edge = db.rtree.nearest((start_node[1],start_node[2],start_node[1],start_node[2]),1)
end_edge = db.rtree.nearest((end_node[1],end_node[2],end_node[1],end_node[2]),1)
start = db.get_nodes_from_edge("EDGES",list(start_edge)[0])
end = db.get_nodes_from_edge("EDGES",list(end_edge)[0])
print(G.number_of_nodes())
print(G.number_of_edges())
print(start[0],end[0])
shortest_nodes = shortest_path(G, start[0], end[0])
print(shortest_nodes)
for i in shortest_nodes:
  print(db.get_by_id('NODES',i))

    


176716
184485
176716
179983
9205092445 1789741152
[9205092445, 1302793233, 1302793202, 1302793100, 1302793260, 8041554207, 8041554208, 8041554213, 8041554212, 8041554215, 8041554216, 8038742596, 38021812, 8038742601, 8041311482, 38021818, 8474389930, 38021822, 38021825, 8041554190, 38021832, 38021835, 38021839, 37991778, 7357739536, 7357739539, 4033858345, 8461702808, 7155793773, 4033858326, 7155795887, 8049688924, 38021845, 8114146962, 8114146961, 8114146972, 8114146971, 8114146970, 8114146969, 8114146968, 7262924648, 7262924637, 7262924636, 37995074, 878767393, 8097743010, 878767404, 878767383, 878766933, 878766947, 878766951, 878766966, 878766971, 878766979, 878766990, 878766994, 878767001, 6140426297, 878767012, 8659282581, 8659282580, 8659282579, 8659282578, 8659282577, 8659282576, 8659282575, 878227488, 38019245, 38037433, 38001179, 38037432, 38009664, 9610028944, 9610028943, 9610077747, 1789741153, 1789741152]
(9205092445, -86.8653957, 40.4743809)
(1302793233, -86.8658199, 40.47